# Chapter 21: Debugging Techniques

Debugging is the systematic process of finding and fixing errors in code. Python provides
a rich set of tools for understanding what went wrong, including the `traceback` module,
the `pdb` debugger, the `warnings` module, and `assert` statements. This notebook covers
these tools and practical strategies for efficient debugging.

## Topics Covered
- **traceback module**: `format_exc()`, `extract_stack()`, `print_exception()`
- **Exception chaining**: `__cause__` and `__context__`
- **breakpoint()**: The built-in debugger hook and `PYTHONBREAKPOINT`
- **pdb commands**: `n`, `s`, `c`, `p`, `l`, `w`, `b` (conceptual overview)
- **Post-mortem debugging**: Debugging after an exception
- **warnings module**: `warn()`, `filterwarnings()`, warning categories
- **assert statements**: When and when not to use them
- **Debugging strategies**: Binary search, rubber duck, logging-based

## The traceback Module

The `traceback` module provides utilities for extracting, formatting, and printing
stack traces. This is essential for logging exceptions, sending error reports, or
building custom error handling.

Key functions:
- `traceback.format_exc()` -- Returns the current exception's traceback as a string
- `traceback.print_exception()` -- Prints exception info to a file (default: stderr)
- `traceback.extract_stack()` -- Returns the current call stack as a list of frames
- `traceback.extract_tb()` -- Extracts frames from a traceback object
- `traceback.format_exception()` -- Formats exception info as a list of strings

In [ ]:
import traceback
import sys


def level_three() -> None:
    """The deepest function that raises an exception."""
    data: dict[str, int] = {"a": 1, "b": 2}
    return data["missing_key"]  # KeyError!


def level_two() -> None:
    """An intermediate function."""
    level_three()


def level_one() -> None:
    """The top-level function."""
    level_two()


# traceback.format_exc() -- capture traceback as a string
print("=== traceback.format_exc() ===")
try:
    level_one()
except KeyError:
    tb_string: str = traceback.format_exc()
    print(tb_string)

# traceback.print_exception() -- print to stdout instead of stderr
print("=== traceback.print_exception() ===")
try:
    level_one()
except KeyError:
    exc_type, exc_value, exc_tb = sys.exc_info()
    traceback.print_exception(exc_type, exc_value, exc_tb, file=sys.stdout)

In [ ]:
import traceback


def show_stack() -> None:
    """Demonstrate extract_stack() to inspect the call stack."""
    print("=== traceback.extract_stack() ===")
    stack = traceback.extract_stack()
    for frame in stack:
        print(f"  File: {frame.filename}")
        print(f"  Line: {frame.lineno}, in {frame.name}")
        if frame.line:
            print(f"  Code: {frame.line}")
        print()


def caller() -> None:
    """Calls show_stack to demonstrate the call chain."""
    show_stack()


caller()

# extract_tb() works with traceback objects from exceptions
print("=== traceback.extract_tb() ===")
try:
    result: float = 1 / 0
except ZeroDivisionError:
    import sys
    _, _, tb = sys.exc_info()
    frames = traceback.extract_tb(tb)
    for frame in frames:
        print(f"  {frame.filename}:{frame.lineno} in {frame.name}")
        print(f"    {frame.line}")

## Exception Chaining: `__cause__` and `__context__`

When one exception leads to another, Python tracks the relationship through two
attributes:

- **`__cause__`** (explicit chaining) -- Set by `raise X from Y`. Indicates that `Y`
  directly caused `X`.
- **`__context__`** (implicit chaining) -- Automatically set when an exception is raised
  while handling another exception.
- **`__suppress_context__`** -- Set to `True` by `raise X from None` to hide the
  implicit context.

In [ ]:
import traceback


# Explicit chaining with 'raise ... from ...'
class DatabaseError(Exception):
    """Error accessing the database."""


class ServiceError(Exception):
    """Error in the service layer."""


def query_database(query: str) -> list[dict]:
    """Simulate a database query that fails."""
    raise DatabaseError(f"Connection refused for query: {query}")


def get_users() -> list[dict]:
    """Service function that wraps database errors."""
    try:
        return query_database("SELECT * FROM users")
    except DatabaseError as db_err:
        # Explicit chaining: ServiceError caused by DatabaseError
        raise ServiceError("Failed to fetch users") from db_err


print("=== Explicit chaining (raise ... from ...) ===")
try:
    get_users()
except ServiceError as e:
    print(f"Caught: {e}")
    print(f"  __cause__:  {e.__cause__}")
    print(f"  __context__: {e.__context__}")
    print()
    # Full traceback shows the chain
    traceback.print_exc()

In [ ]:
import traceback


# Implicit chaining: exception raised while handling another
def implicit_chain_example() -> None:
    """Demonstrate implicit exception chaining."""
    try:
        int("not_a_number")
    except ValueError:
        # This raises a new exception WHILE handling ValueError
        data: dict[str, int] = {}
        _ = data["missing"]  # KeyError with implicit context


print("=== Implicit chaining (__context__) ===")
try:
    implicit_chain_example()
except KeyError as e:
    print(f"Caught: KeyError({e})")
    print(f"  __cause__:  {e.__cause__}")    # None (not explicit)
    print(f"  __context__: {e.__context__}")  # The original ValueError


# Suppressing context with 'raise ... from None'
def suppressed_context_example() -> None:
    """Suppress the original exception context."""
    try:
        int("not_a_number")
    except ValueError:
        raise RuntimeError("Invalid input provided") from None


print("\n=== Suppressed context (raise ... from None) ===")
try:
    suppressed_context_example()
except RuntimeError as e:
    print(f"Caught: {e}")
    print(f"  __cause__:             {e.__cause__}")
    print(f"  __suppress_context__:  {e.__suppress_context__}")
    print(f"  __context__:           {e.__context__}")  # Still set, but suppressed

## breakpoint() and PYTHONBREAKPOINT

The `breakpoint()` built-in (Python 3.7+) is the standard way to drop into an
interactive debugger. By default, it calls `pdb.set_trace()`, but you can customize
its behavior through the `PYTHONBREAKPOINT` environment variable:

| `PYTHONBREAKPOINT` value | Behavior |
|--------------------------|----------|
| *(not set)* | Calls `pdb.set_trace()` (default) |
| `"0"` | Disables all breakpoints (no-op) |
| `"ipdb.set_trace"` | Uses ipdb instead of pdb |
| `"pudb.set_trace"` | Uses pudb (full-screen debugger) |
| Any callable path | Calls that function |

**Note**: We will not actually invoke `breakpoint()` in this notebook since it would
pause execution and require interactive input. The examples below are conceptual.

In [ ]:
import os
import sys

# Show the current PYTHONBREAKPOINT setting
current_setting: str = os.environ.get("PYTHONBREAKPOINT", "(not set -- defaults to pdb)")
print(f"Current PYTHONBREAKPOINT: {current_setting}")

# The breakpoint() hook function
print(f"sys.breakpointhook: {sys.breakpointhook}")

# How you would use breakpoint() in practice:
print("\n--- Example usage (conceptual, not executed) ---")
print("""
def process_data(items: list[str]) -> list[str]:
    results = []
    for item in items:
        transformed = item.strip().lower()
        breakpoint()  # Execution pauses here; you can inspect variables
        results.append(transformed)
    return results
""")

# Disabling breakpoints via environment variable
print("To disable all breakpoints:")
print("  export PYTHONBREAKPOINT=0")
print("  python my_script.py")

# Using a different debugger
print("\nTo use ipdb instead:")
print("  export PYTHONBREAKPOINT=ipdb.set_trace")
print("  python my_script.py")

## pdb Commands Overview

The Python Debugger (`pdb`) provides an interactive command-line interface for stepping
through code, inspecting variables, and setting breakpoints. Here are the essential
commands:

| Command | Full Name | Description |
|---------|-----------|------------|
| `n` | next | Execute the next line (step **over** function calls) |
| `s` | step | Step **into** the next function call |
| `c` | continue | Continue execution until the next breakpoint |
| `p expr` | print | Print the value of an expression |
| `pp expr` | pretty-print | Pretty-print the value of an expression |
| `l` | list | Show source code around the current line |
| `ll` | longlist | Show the entire current function |
| `w` | where | Print the full stack trace |
| `b N` | break | Set a breakpoint at line N |
| `b func` | break | Set a breakpoint at a function |
| `cl N` | clear | Clear breakpoint number N |
| `r` | return | Continue until the current function returns |
| `q` | quit | Quit the debugger |
| `h` | help | Show help for commands |
| `!stmt` | execute | Execute a Python statement |

In [ ]:
# Conceptual pdb session walkthrough
# This shows what a typical debugging session looks like

sample_code: str = """
def find_average(numbers: list[float]) -> float:
    total = sum(numbers)
    count = len(numbers)
    average = total / count    # BUG: crashes if numbers is empty
    return average

# Running with pdb:
# $ python -m pdb my_script.py

# Or with breakpoint():
# breakpoint()
# result = find_average([])
"""

pdb_session: str = """
Typical pdb session:

> my_script.py(2)find_average()
-> total = sum(numbers)
(Pdb) p numbers          # Print the argument
[]
(Pdb) n                  # Step to next line
> my_script.py(3)find_average()
-> count = len(numbers)
(Pdb) n                  # Step to next line
> my_script.py(4)find_average()
-> average = total / count
(Pdb) p total, count     # Inspect variables
(0, 0)
(Pdb) p count == 0       # Evaluate expressions
True
(Pdb) w                  # Show call stack
  my_script.py(8)<module>()
-> result = find_average([])
> my_script.py(4)find_average()
-> average = total / count
(Pdb) l                  # List source code
  1     def find_average(numbers: list[float]) -> float:
  2         total = sum(numbers)
  3         count = len(numbers)
  4  ->     average = total / count
  5         return average
(Pdb) q                  # Quit debugger
"""

print("Sample code with a bug:")
print(sample_code)
print(pdb_session)

## Post-Mortem Debugging

**Post-mortem debugging** lets you inspect the program state at the point where an
unhandled exception occurred. Instead of setting breakpoints beforehand, you debug
**after** the crash.

- `pdb.post_mortem(tb)` -- Start pdb at the frame where the exception occurred
- `pdb.pm()` -- Shortcut that uses `sys.last_traceback`
- `python -m pdb script.py` -- Automatically enters post-mortem on crash

In [ ]:
import sys
import traceback


def buggy_function(data: dict[str, int]) -> int:
    """A function with a bug we want to debug post-mortem."""
    total: int = 0
    for key in ["x", "y", "z"]:
        total += data[key]  # KeyError if key is missing
    return total


# Capture the exception for post-mortem analysis
print("=== Capturing exception for post-mortem analysis ===")
try:
    result = buggy_function({"x": 10, "y": 20})  # Missing "z"
except KeyError:
    exc_type, exc_value, exc_tb = sys.exc_info()

    # In a real debugging session, you would call:
    # import pdb; pdb.post_mortem(exc_tb)
    # This drops you into pdb at the exact line that crashed

    # For this notebook, we inspect the traceback programmatically
    print(f"Exception type: {exc_type.__name__}")
    print(f"Exception value: {exc_value}")
    print(f"\nTraceback frames:")
    for frame_info in traceback.extract_tb(exc_tb):
        print(f"  {frame_info.name}() at line {frame_info.lineno}")
        print(f"    Code: {frame_info.line}")

    # Access local variables from the crashed frame
    if exc_tb is not None:
        crashed_frame = exc_tb.tb_next  # The frame where the error occurred
        if crashed_frame is not None:
            local_vars = crashed_frame.tb_frame.f_locals
            print(f"\nLocal variables at crash point:")
            for var_name, var_value in local_vars.items():
                print(f"  {var_name} = {var_value!r}")

print("\nPost-mortem usage:")
print("  import pdb; pdb.post_mortem(exc_tb)  # Interactive debugging")
print("  python -m pdb script.py               # Auto post-mortem on crash")

## The warnings Module

The `warnings` module issues alerts about potential issues that are not errors. Warnings
are useful for deprecation notices, questionable coding patterns, and other situations
where you want to notify developers without crashing the program.

Warning categories:

| Category | Purpose |
|----------|--------|
| `UserWarning` | Default category for user-issued warnings |
| `DeprecationWarning` | Feature will be removed in a future version |
| `FutureWarning` | Behavior will change in a future version |
| `PendingDeprecationWarning` | Feature will be deprecated soon |
| `RuntimeWarning` | Questionable runtime behavior |
| `SyntaxWarning` | Questionable syntax |
| `ResourceWarning` | Resource usage issues (unclosed files, etc.) |

In [ ]:
import warnings

# Reset warning filters for this demo
warnings.resetwarnings()

# Basic warning
print("=== Basic warnings ===")
warnings.warn("This is a default UserWarning")
warnings.warn("This function is deprecated", DeprecationWarning, stacklevel=1)
warnings.warn("Behavior will change in v3.0", FutureWarning)

# Custom warning category
class SecurityWarning(UserWarning):
    """Warning about potential security issues."""

warnings.warn("Using default encryption key", SecurityWarning)

In [ ]:
import warnings

# filterwarnings() controls which warnings are shown
# Actions: 'default', 'ignore', 'always', 'error', 'once', 'module'

warnings.resetwarnings()

# 'ignore' suppresses warnings
print("=== Ignoring DeprecationWarning ===")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.warn("This deprecation warning is hidden", DeprecationWarning)
print("(No output -- DeprecationWarning is ignored)")

# 'error' turns warnings into exceptions
print("\n=== Turning warnings into errors ===")
warnings.resetwarnings()
warnings.filterwarnings("error", category=RuntimeWarning)

try:
    warnings.warn("Suspicious runtime behavior", RuntimeWarning)
except RuntimeWarning as e:
    print(f"Caught as exception: {e}")

# 'always' shows the warning every time (default shows once per location)
print("\n=== 'always' filter ===")
warnings.resetwarnings()
warnings.filterwarnings("always", category=UserWarning)

for i in range(3):
    warnings.warn(f"Repeated warning #{i}")

# Filter by message pattern (regex)
print("\n=== Filter by message pattern ===")
warnings.resetwarnings()
warnings.filterwarnings("ignore", message=".*deprecated.*")
warnings.warn("This is deprecated")     # Ignored (matches pattern)
warnings.warn("This is important")       # Shown

warnings.resetwarnings()

## Assert Statements: When and When Not to Use Them

The `assert` statement checks a condition and raises `AssertionError` if it is `False`.
Assertions are a debugging aid, **not** a mechanism for input validation or error handling.

```python
assert condition, "optional error message"
```

**Critical**: Assertions are removed when Python runs with optimization (`-O` flag).
Never use assertions for:
- Input validation
- Security checks
- Conditions that might legitimately fail in production

In [ ]:
# GOOD uses of assert: checking internal invariants and preconditions

def calculate_average(values: list[float]) -> float:
    """Calculate the average. Caller must ensure non-empty list."""
    # Internal precondition: the caller guarantees a non-empty list
    assert len(values) > 0, "Internal error: values must not be empty"
    return sum(values) / len(values)


def normalize(data: list[float]) -> list[float]:
    """Normalize data to [0, 1] range."""
    min_val: float = min(data)
    max_val: float = max(data)
    spread: float = max_val - min_val

    assert spread >= 0, f"Internal error: negative spread {spread}"

    if spread == 0:
        return [0.5] * len(data)

    result: list[float] = [(x - min_val) / spread for x in data]

    # Post-condition: all values should be in [0, 1]
    assert all(0.0 <= v <= 1.0 for v in result), "Internal error: normalization failed"

    return result


# Good: assert catches internal bugs
print("Average:", calculate_average([10.0, 20.0, 30.0]))
print("Normalized:", normalize([2.0, 5.0, 8.0, 11.0]))

# Triggering an assertion
print("\n--- Triggering an assertion ---")
try:
    calculate_average([])  # Violates precondition
except AssertionError as e:
    print(f"AssertionError: {e}")

In [ ]:
# BAD uses of assert (NEVER do this in production code)

# BAD: Using assert for input validation
def bad_validate_age(age: int) -> None:
    """BAD: Assertions are removed with -O flag!"""
    assert isinstance(age, int), "Age must be an integer"  # WRONG
    assert 0 <= age <= 150, "Age must be between 0 and 150"  # WRONG


# GOOD: Using proper exceptions for input validation
def good_validate_age(age: int) -> None:
    """GOOD: Proper validation that works even with -O flag."""
    if not isinstance(age, int):
        raise TypeError(f"Age must be an integer, got {type(age).__name__}")
    if not 0 <= age <= 150:
        raise ValueError(f"Age must be between 0 and 150, got {age}")


# BAD: Assert with side effects (the tuple form is always truthy!)
print("=== Common assert pitfall ===")
x: int = -1

# This is ALWAYS True because a non-empty tuple is truthy
assert (x > 0, "x must be positive")  # BUG: This never fails!
print(f"x = {x} (the assert above did NOT catch x < 0!)")

# Correct form (no parentheses around the condition + message):
try:
    assert x > 0, "x must be positive"
except AssertionError as e:
    print(f"Correct assertion caught: {e}")

print("\n--- Summary ---")
print("USE assert for: internal invariants, postconditions, debug checks")
print("AVOID assert for: input validation, security, any side-effect logic")

## Debugging Strategies

Beyond tools, effective debugging requires systematic strategies:

1. **Reproduce first** -- Create the smallest input that triggers the bug
2. **Read the error message** -- Python tracebacks are detailed and informative
3. **Binary search** -- Narrow down the problem by testing at the midpoint
4. **Rubber duck debugging** -- Explain the code line-by-line to an inanimate object
5. **Logging-based debugging** -- Add targeted log statements to trace execution
6. **Simplify** -- Remove complexity until the bug disappears, then add it back

In [ ]:
import logging
import sys

# Strategy: Logging-based debugging
# Add temporary logging to trace execution flow and variable state

# Set up a debug logger
debug_logger = logging.getLogger("debug")
debug_logger.setLevel(logging.DEBUG)
debug_logger.handlers.clear()
debug_logger.propagate = False

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter("  DEBUG | %(message)s"))
debug_logger.addHandler(handler)


def merge_sorted(left: list[int], right: list[int]) -> list[int]:
    """Merge two sorted lists into one sorted list."""
    debug_logger.debug("merge_sorted called: left=%s, right=%s", left, right)

    result: list[int] = []
    i: int = 0
    j: int = 0

    while i < len(left) and j < len(right):
        debug_logger.debug("  comparing left[%d]=%d vs right[%d]=%d", i, left[i], j, right[j])
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    # Append remaining elements
    result.extend(left[i:])
    result.extend(right[j:])

    debug_logger.debug("  result=%s", result)
    return result


# Run the function with logging enabled
print("=== Logging-based debugging ===")
merged = merge_sorted([1, 3, 5, 7], [2, 4, 6, 8])
print(f"\nFinal result: {merged}")

# To disable debug logging without removing code:
print("\n=== Same function, logging disabled ===")
debug_logger.setLevel(logging.WARNING)  # Suppress DEBUG messages
merged = merge_sorted([10, 20], [15, 25])
print(f"Result: {merged}")

# Clean up
debug_logger.removeHandler(handler)

In [ ]:
# Strategy: Binary search debugging
# When you have a large codebase or data pipeline, narrow down the bug
# by testing at the midpoint of the suspected problem area.

def pipeline_step_1(data: list[int]) -> list[int]:
    """Filter out negative numbers."""
    return [x for x in data if x >= 0]


def pipeline_step_2(data: list[int]) -> list[int]:
    """Double each value."""
    return [x * 2 for x in data]


def pipeline_step_3(data: list[int]) -> list[int]:
    """BUG: Accidentally squares instead of adding 1."""
    return [x ** 2 for x in data]  # Should be [x + 1 for x in data]


def pipeline_step_4(data: list[int]) -> list[int]:
    """Sort the results."""
    return sorted(data)


def run_pipeline(data: list[int]) -> list[int]:
    """Run a multi-step data pipeline."""
    result = data
    result = pipeline_step_1(result)
    result = pipeline_step_2(result)
    result = pipeline_step_3(result)
    result = pipeline_step_4(result)
    return result


# We know the output is wrong, but which step has the bug?
input_data: list[int] = [3, -1, 2, -5, 4]
expected: list[int] = [5, 5, 7, 9]  # filter -> double -> add1 -> sort
actual: list[int] = run_pipeline(input_data)
print(f"Input:    {input_data}")
print(f"Expected: {expected}")
print(f"Actual:   {actual}")
print(f"Match:    {expected == actual}")

# Binary search: check the midpoint (after step 2)
print("\n--- Binary search debugging ---")
after_step_1: list[int] = pipeline_step_1(input_data)
print(f"After step 1 (filter):  {after_step_1}")  # [3, 2, 4] -- correct

after_step_2: list[int] = pipeline_step_2(after_step_1)
print(f"After step 2 (double):  {after_step_2}")  # [6, 4, 8] -- correct

after_step_3: list[int] = pipeline_step_3(after_step_2)
print(f"After step 3 (add 1?):  {after_step_3}")  # [36, 16, 64] -- BUG HERE!
print(f"  Expected after step 3: [7, 5, 9]")
print(f"  Bug found: step 3 squares instead of adding 1!")

## Summary

### Key Takeaways

| Tool | Purpose | When to Use |
|------|---------|------------|
| **traceback** | Format and inspect stack traces | Logging exceptions, error reports |
| **Exception chaining** | `raise X from Y`, `__cause__`, `__context__` | Wrapping exceptions while preserving the cause |
| **breakpoint()** | Drop into interactive debugger | Interactive investigation of live state |
| **pdb** | Step through code, inspect variables | Complex bugs requiring line-by-line analysis |
| **Post-mortem** | Debug after an exception | When the crash is hard to reproduce |
| **warnings** | Non-fatal alerts about potential issues | Deprecations, questionable patterns |
| **assert** | Check internal invariants | Debug-time verification of assumptions |

### Debugging Strategy Checklist
1. Read the traceback carefully -- the answer is often in the error message
2. Reproduce the bug with the smallest possible input
3. Use `logging` to trace execution flow without stopping the program
4. Use `breakpoint()` and pdb when you need to inspect live state interactively
5. Use binary search to narrow down which step or component contains the bug
6. Use `assert` for internal invariants, but never for input validation
7. Use `warnings.warn()` for deprecations rather than `print()` or logging
8. Explain the code out loud (rubber duck debugging) when you are stuck